In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import os
import ast
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.patches as mpatches
import seaborn as sn
from tqdm.std import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
class Config:
    data_dir = '../data/'
    working_dir = '../src/'
    models_dir = '../models/'
    submissions_dir = '../submissions'

In [3]:
from google.colab import files
  
  
uploaded = files.upload()
import pandas as pd
import io

  
train = pd.read_csv(io.BytesIO(uploaded['Train.csv']))

Saving Train.csv to Train (2).csv


In [4]:
uploaded = files.upload()
metadata = pd.read_csv(io.BytesIO(uploaded['metadata.csv']))

Saving metadata.csv to metadata (2).csv


In [5]:
uploaded = files.upload()
test = pd.read_csv(io.BytesIO(uploaded['Test.csv']))

Saving Test.csv to Test (2).csv


In [6]:
uploaded = files.upload()
ss = pd.read_csv(io.BytesIO(uploaded['SampleSubmission.csv']))

Saving SampleSubmission.csv to SampleSubmission (2).csv


train = pd.read_csv(filepath_or_buffer=os.path.join(Config.data_dir, "Train.csv"))
metadata = pd.read_csv(filepath_or_buffer=os.path.join(Config.data_dir, "metadata.csv"))
test = pd.read_csv(filepath_or_buffer=os.path.join(Config.data_dir, "Test.csv"))
ss = pd.read_csv(filepath_or_buffer=os.path.join(Config.data_dir, "SampleSubmission.csv"))



In [7]:
print(metadata.isna().sum())

ID                         0
RegistrationDate           0
Deposit                    0
UpsellDate             36370
AccessoryRate              0
PaymentMethod              0
rateTypeEntity             0
RatePerUnit                0
DaysOnDeposit              0
MainApplicantGender        0
Age                     6939
Region                  1934
Town                       0
Occupation                 0
SupplierName               0
Term                       0
TotalContractValue         0
ExpectedTermDate           0
FirstPaymentDate           0
LastPaymentDate            0
dtype: int64


In [8]:
metadata[metadata.isna().Region].head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
3,ID_HXBJFHB,11/25/2015 0:00,2000,NaN,0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740,11/23/2016 0:00,11/25/2015 13:25,5/22/2017 16:46
17,ID_AYIBQUP,12/12/2015 0:00,2000,NaN,0,FINANCED,DAILY,35,7,Male,44.0,NaN,UNKNOWN,Other,d_light,364,14740,12/10/2016 0:00,12/12/2015 7:00,9/2/2020 20:30
51,ID_35MNQL1,12/6/2015 0:00,2000,NaN,0,FINANCED,DAILY,35,7,Male,42.0,NaN,UNKNOWN,Farmer,d_light,364,14740,12/4/2016 0:00,12/7/2015 6:21,11/18/2017 11:07
84,ID_RWA0Q3U,7/12/2017 14:44,2000,NaN,0,FINANCED,DAILY,40,7,Female,52.0,NaN,UNKNOWN,Government Employee,d_light,364,16560,7/11/2018 14:44,7/12/2017 14:45,9/2/2020 20:31
85,ID_QWI7WWN,11/24/2015 0:00,2000,NaN,0,FINANCED,DAILY,35,7,Male,40.0,NaN,UNKNOWN,Other,d_light,364,14740,11/22/2016 0:00,11/24/2015 14:02,11/25/2016 14:58


In [9]:
metadata.Region.value_counts()

Nyanza                6998
North Rift            5344
Nairobi Region        5056
South Rift            4759
Coast Region          4671
Western               4508
Mount Kenya Region    4073
Name: Region, dtype: int64

In [10]:
pd.merge(left=metadata[metadata.isna().Region], right=test, on="ID").head(2)
#test data also contains empty values in region therefore we need to treat as a category.

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_UOT1MF3,7/13/2017 8:56,2000,NaN,0,FINANCED,DAILY,40,7,Male,54.0,NaN,UNKNOWN,Business,d_light,364,16560,7/12/2018 8:56,7/13/2017 8:56,9/2/2020 20:31,"['07-2017', '08-2017', '09-2017', '10-2017', '11-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '10-2018', '11-2018', '12-2018', '01-2019', '02-2019', '03-2019', '04-2019', '07-2019']","[2720.0, 1080.0, 1200.0, 1000.0, 960.0, 1160.0, 1120.0, 600.0, 1040.0, 840.0, 680.0, 640.0, 680.0, 520.0, 80.0, 80.0, 40.0, 40.0, 240.0, 40.0, 80.0, 40.0]"
1,ID_290EB8N,7/5/2017 9:06,2000,NaN,0,FINANCED,DAILY,40,7,Male,31.0,NaN,UNKNOWN,Labourer,d_light,364,16560,7/4/2018 9:06,7/5/2017 8:54,9/2/2020 20:35,"['07-2017', '08-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018', '10-2018']","[2710.0, 120.0, 200.0, 300.0, 50.0, 250.0, 550.0, 100.0, 500.0, 350.0, 350.0, 950.0, 200.0]"


In [11]:
metadata.Region.fillna(value="Other", inplace=True)

In [12]:
metadata.UpsellDate.fillna(value=0, inplace=True)

In [13]:
pd.merge(left=metadata[metadata.isna().Age], right=test, on="ID").head(2)
##now just using mean here to remove 0 in metadata.


,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory
0,ID_VJ80SX2,12/14/2015 0:00,2000,0,0,FINANCED,DAILY,35,7,Female,NaN,Mount Kenya Region,Embu,Business,d_light,364,14740,12/12/2016 0:00,12/14/2015 14:20,12/12/2016 11:59,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016']","[3000.0, 850.0, 750.0, 1500.0, 650.0, 1250.0, 1000.0]"
1,ID_ZLW8XIB,12/5/2015 0:00,2000,0,0,FINANCED,DAILY,35,7,Male,NaN,Mount Kenya Region,Embu,Farmer,d_light,364,14740,12/3/2016 0:00,12/5/2015 15:28,4/1/2017 19:19,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016']","[3200.0, 735.0, 1320.0, 1000.0, 1170.0, 866.0, 730.0, 100.0, 100.0, 450.0, 700.0]"


In [14]:
metadata.Age.fillna(value=round(metadata.Age.mean()), inplace=True)

In [15]:
merged = pd.merge(left=metadata, right=pd.concat(objs=[train, test]).fillna(value=0), on="ID")
merged.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_K00S4N4,12/10/2015 0:00,2000,0,0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740,12/8/2016 0:00,12/10/2015 9:52,10/23/2016 4:52,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016']","[3050.0, 1050.0, 910.0, 1050.0, 1050.0]",1225.0,1050.0,1190.0,525.0,1750.0,1890.0
1,ID_6L67PAA,12/9/2015 0:00,2000,0,0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740,12/7/2016 0:00,12/9/2015 13:14,5/24/2020 15:32,"['12-2015', '01-2016', '02-2016', '03-2016', '05-2016', '07-2016']","[4000.0, 1050.0, 1050.0, 1050.0, 1050.0, 400.0]",0.0,0.0,0.0,0.0,0.0,0.0
2,ID_102CV85,12/18/2015 0:00,2000,3/29/2018 10:14,35,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480,1/13/2017 0:00,12/18/2015 6:22,2/1/2017 15:23,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016']","[4245.0, 980.0, 735.0, 735.0, 1470.0, 735.0, 980.0, 915.0, 735.0]",980.0,980.0,1225.0,980.0,935.0,355.0
3,ID_HXBJFHB,11/25/2015 0:00,2000,0,0,FINANCED,DAILY,35,7,Female,43.0,Other,UNKNOWN,Teacher,d_light,364,14740,11/23/2016 0:00,11/25/2015 13:25,5/22/2017 16:46,"['11-2015', '12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '10-2016']","[2245.0, 980.0, 980.0, 1225.0, 980.0, 980.0, 980.0, 1225.0, 735.0, 490.0, 250.0]",250.0,1000.0,250.0,500.0,560.0,1150.0
4,ID_3K9VZ5J,12/2/2015 0:00,2000,0,0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740,11/30/2016 0:00,12/5/2015 10:34,5/12/2017 16:50,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016', '11-2016']","[2750.0, 1000.0, 750.0, 1000.0, 955.0, 880.0, 280.0, 665.0, 770.0, 420.0, 525.0, 735.0]",630.0,805.0,700.0,855.0,245.0,775.0


In [16]:
merged.drop(["FirstPaymentDate"],axis=1,inplace=True)

In [17]:
def diff(a,b):
    start=b.split('/')
    end=a.split('/')
    return int(end[1])-int(start[1])+12*(int(end[0])-int(start[0]))

In [18]:
#merged['month']=merged['RegistrationDate']
#merged['year']=merged['RegistrationDate']
merged['time']=merged['RegistrationDate']
merged['PaidAmount']=merged['TotalContractValue']
merged['PeriodDiff']=merged['TotalContractValue']
for i in tqdm(range(0,merged.shape[0]),desc='Row'):
    year=int(merged.RegistrationDate.iloc[i].split('/')[0])
    month=int(merged.RegistrationDate.iloc[i].split('/')[1])
    time=np.ceil(merged.loc[i,'Term']/30)
    merged.loc[i,'time']=time
    if merged.UpsellDate.iloc[i]!=0:
        merged.loc[i,'UpsellDate']=diff(merged.UpsellDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'LastPaymentDate']=diff(merged.LastPaymentDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'ExpectedTermDate']=diff(merged.ExpectedTermDate.iloc[i],merged.RegistrationDate.iloc[i])/time
    merged.loc[i,'m1']=merged.loc[i,'m1']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m2']=merged.loc[i,'m2']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m3']=merged.loc[i,'m3']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m4']=merged.loc[i,'m4']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m5']=merged.loc[i,'m5']/merged.loc[i,'TotalContractValue']
    merged.loc[i,'m6']=merged.loc[i,'m6']/merged.loc[i,'TotalContractValue']
    dates=[]
    for j in merged.loc[i,'TransactionDates'].split("'"):
        if '-' in j:
            date=j.split('-')
            dates.append((int(date[0])-month+12*(int(date[1])-year))/time)
    merged.at[i,'TransactionDates']=dates
    history=[]
    for j in merged.loc[i,'PaymentsHistory'].split("[")[1].split("]")[0].split(","):
        if '.' in j:
            history.append(float(j)/merged.loc[i,'TotalContractValue'])
    merged.at[i,'PaymentsHistory']=history
    merged.loc[i,'PaidAmount']=sum(history)
    merged.loc[i,'PeriodDiff']=merged.loc[i,'LastPaymentDate']-dates[-1]
merged.drop(["RegistrationDate","Term"],axis=1,inplace=True)

Row: 100%|██████████| 37343/37343 [05:29<00:00, 113.17it/s]


In [19]:
merged.tail(2)

,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
37341,ID_GHHAQ9D,2400,0,0,FINANCED,DAILY,50,3,Male,27.0,Other,UNKNOWN,Labourer,d_light,14400,11.625,3.5,"[3024.75, 3024.875, 3025.0, 3025.125, 3025.25, 3025.375, 3025.5, 3025.625, 3025.75]","[0.18194444444444444, 0.09027777777777778, 0.08333333333333333, 0.07291666666666667, 0.0763888888888889, 0.04513888888888889, 0.010416666666666666, 0.0038194444444444443, 0.021875]",0.034722,0.059028,0.045139,0.048611,0.041667,0.048611,8,0.586111,-3022.250
37342,ID_PAI1FJK,2400,0,0,FINANCED,DAILY,50,3,Male,36.0,Coast Region,Mombasa,Farmer,d_light,14400,11.625,5.25,"[3024.75, 3024.875, 3025.125, 3025.25, 3025.375, 3025.5, 3025.625, 3025.75, 3025.875]","[0.1701388888888889, 0.003472222222222222, 0.003472222222222222, 0.09375, 0.03125, 0.07291666666666667, 0.04861111111111111, 0.0798611111111111, 0.05277777777777778]",0.100694,0.055556,0.017361,0.003472,0.010417,0.045139,8,0.556250,-3020.625


In [20]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
category_list=['PaymentMethod','rateTypeEntity','MainApplicantGender','Region','Town','Occupation','SupplierName']
for s in category_list:
    merged[s] = lb_make.fit_transform(merged[s])
    print(lb_make.transform(lb_make.classes_))
    print(lb_make.classes_)
merged.head(2)

[0]
['FINANCED']
[0 1 2]
['DAILY' 'MONTHLY' 'WEEKLY']
[0 1]
['Female' 'Male']
[0 1 2 3 4 5 6 7]
['Coast Region' 'Mount Kenya Region' 'Nairobi Region' 'North Rift'
 'Nyanza' 'Other' 'South Rift' 'Western']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
['Baringo' 'Bomet' 'Bungoma' 'Busia' 'Elgeyo/Marakwet' 'Embu' 'Garissa'
 'Homa Bay' 'Isiolo' 'Kajiado' 'Kakamega' 'Kericho' 'Kiambu' 'Kilifi'
 'Kirinyaga' 'Kisii' 'Kisumu' 'Kitui' 'Kwale' 'Laikipia' 'Lamu' 'Machakos'
 'Makueni' 'Mandera' 'Marsabit' 'Meru' 'Migori' 'Mombasa' 'Muranga'
 'Nairobi City' 'Nakuru' 'Nandi' 'Narok' 'Nyamira' 'Nyandarua' 'Nyeri'
 'Samburu' 'Siaya' 'Taita/Taveta' 'Tana River' 'Tharaka-Nithi'
 'Trans Nzoia' 'Turkana' 'UNKNOWN' 'Uasin Gishu' 'Vihiga' 'Wajir'
 'West Pokot']
[0 1 2 3 4 5 6]
['Business' 'Driver/Motorbike Rider' 'Farmer' 'Government Employee'
 'Labourer' 'Other' 'Teacher']
[0]
['d_light']


,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0,0,0,35,7,1,41.0,1,5,5,0,14740,-0.153846,-0.846154,"[1849.076923076923, 1849.1538461538462, 1849.2307692307693, 1849.3076923076924, 1849.3846153846155]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13,0.482361,-1850.230769
1,ID_6L67PAA,2000,0,0,0,0,35,7,1,33.0,0,13,5,0,14740,-0.153846,-5.30769,"[1849.1538461538462, 1849.2307692307693, 1849.3076923076924, 1849.3846153846155, 1849.5384615384614, 1849.6923076923076]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,0.583446,-1855.000000


In [21]:
## As these two variable has only one class.
merged.drop(['PaymentMethod','SupplierName'],axis=1,inplace=True)

In [22]:
merged.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0,0,35,7,1,41.0,1,5,5,14740,-0.153846,-0.846154,"[1849.076923076923, 1849.1538461538462, 1849.2307692307693, 1849.3076923076924, 1849.3846153846155]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13,0.482361,-1850.230769
1,ID_6L67PAA,2000,0,0,0,35,7,1,33.0,0,13,5,14740,-0.153846,-5.30769,"[1849.1538461538462, 1849.2307692307693, 1849.3076923076924, 1849.3846153846155, 1849.5384615384614, 1849.6923076923076]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13,0.583446,-1855.000000
2,ID_102CV85,2000,-6.92857,35,0,35,7,0,48.0,2,22,0,29480,-9.78571,-9.78571,"[1716.4285714285713, 1716.5, 1716.5714285714287, 1716.642857142857, 1716.7142857142858, 1716.7857142857142, 1716.857142857143, 1716.9285714285713, 1717.0]","[0.14399592944369063, 0.03324287652645862, 0.024932157394843963, 0.024932157394843963, 0.049864314789687926, 0.024932157394843963, 0.03324287652645862, 0.031037991858887382, 0.024932157394843963]",0.033243,0.033243,0.041554,0.033243,0.031716,0.012042,14,0.391113,-1726.785714
3,ID_HXBJFHB,2000,0,0,0,35,7,0,43.0,5,43,6,14740,-0.153846,-5.76923,"[1848.7692307692307, 1848.8461538461538, 1848.923076923077, 1849.0, 1849.076923076923, 1849.1538461538462, 1849.2307692307693, 1849.3076923076924, 1849.3846153846155, 1849.4615384615386, 1849.6153846153845]","[0.1523066485753053, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.06648575305291723, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.049864314789687926, 0.03324287652645862, 0.016960651289009497]",0.016961,0.067843,0.016961,0.033921,0.037992,0.078019,13,0.751018,-1855.384615
4,ID_3K9VZ5J,2000,0,0,0,35,7,0,56.0,1,14,5,14740,1.23077,-5.69231,"[1849.6923076923076, 1849.7692307692307, 1849.8461538461538, 1849.923076923077, 1850.0, 1850.076923076923, 1850.1538461538462, 1850.2307692307693, 1850.3076923076924, 1850.3846153846155, 1850.4615384615386, 1850.5384615384614]","[0.1865671641791045, 0.06784260515603799, 0.05088195386702849, 0.06784260515603799, 0.06478968792401628, 0.05970149253731343, 0.018995929443690638, 0.04511533242876527, 0.05223880597014925, 0.028493894165535955, 0.03561736770691994, 0.049864314789687926]",0.042741,0.054613,0.047490,0.058005,0.016621,0.052578,13,0.727951,-1856.230769


In [23]:
merged.to_csv("Final.csv",index=False)

##Advance:: Can use clustering/estimation to get age.
##Advance:: Care for Days on deposit(Some finance has to be read)
##Advance:: Care for deposit in paid amount.

In [24]:

from tsfresh import extract_features
df=pd.DataFrame(columns=["time","money","id"])
for k in tqdm(range(0,merged.shape[0])):
    df_temp=pd.DataFrame({"time":merged.loc[k,"TransactionDates"],"money":merged.loc[k,"PaymentsHistory"]})
    df_temp['id']=merged.loc[k,"ID"]
    df=pd.concat([df, df_temp], ignore_index=True)
print(df.shape)

100%|██████████| 37343/37343 [02:35<00:00, 240.25it/s]

(607851, 3)


In [25]:
df_final=df.copy()

In [26]:
df_final=extract_features(df_final,column_id='id',column_sort="time").dropna(axis=1)

Feature Extraction: 100%|██████████| 5/5 [36:16<00:00, 435.34s/it]


In [27]:
print(df_final.shape)


(37343, 268)


In [28]:
df_final.head(5)

,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_coef

In [29]:
df.sort_values(by='id',inplace=True)

In [30]:
df.head(5)

,time,money,id
68925,1856.153846,0.028986,ID_000RHRU
68927,1856.307692,0.042271,ID_000RHRU
68926,1856.230769,0.038647,ID_000RHRU
68930,1856.846154,0.007246,ID_000RHRU
68924,1856.076923,0.045894,ID_000RHRU


In [31]:
ids=df['id']

In [32]:
ids.drop_duplicates(inplace=True)

In [33]:
ids.shape

(37343,)

In [34]:
df_final['ID']=ids.values

In [35]:
df_final.head(5)

,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quantile__q_0.6,money__index_mass_quantile__q_0.7,money__index_mass_quantile__q_0.8,money__index_mass_quantile__q_0.9,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","money__cwt_coef

In [36]:
merged1= pd.merge(left=merged, right=df_final.fillna(value=0),on='ID')

In [37]:
merged1.shape

(37343, 294)

In [38]:
merged1.head()

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff,money__variance_larger_than_standard_deviation,money__has_duplicate_max,money__has_duplicate_min,money__has_duplicate,money__sum_values,money__abs_energy,money__mean_abs_change,money__mean_change,money__mean_second_derivative_central,money__median,money__mean,money__length,money__standard_deviation,money__variation_coefficient,money__variance,money__skewness,money__kurtosis,money__root_mean_square,money__absolute_sum_of_changes,money__longest_strike_below_mean,money__longest_strike_above_mean,money__count_above_mean,money__count_below_mean,money__last_location_of_maximum,money__first_location_of_maximum,money__last_location_of_minimum,money__first_location_of_minimum,money__percentage_of_reoccurring_values_to_all_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__sum_of_reoccurring_values,money__sum_of_reoccurring_data_points,money__ratio_value_number_to_time_series_length,money__maximum,money__minimum,money__time_reversal_asymmetry_statistic__lag_1,money__time_reversal_asymmetry_statistic__lag_2,money__time_reversal_asymmetry_statistic__lag_3,money__c3__lag_1,money__c3__lag_2,money__c3__lag_3,money__cid_ce__normalize_True,money__cid_ce__normalize_False,money__symmetry_looking__r_0.0,money__symmetry_looking__r_0.05,money__symmetry_looking__r_0.1,money__symmetry_looking__r_0.15000000000000002,money__symmetry_looking__r_0.2,money__symmetry_looking__r_0.25,money__symmetry_looking__r_0.30000000000000004,money__symmetry_looking__r_0.35000000000000003,money__symmetry_looking__r_0.4,money__symmetry_looking__r_0.45,money__symmetry_looking__r_0.5,money__symmetry_looking__r_0.55,money__symmetry_looking__r_0.6000000000000001,money__symmetry_looking__r_0.65,money__symmetry_looking__r_0.7000000000000001,money__symmetry_looking__r_0.75,money__symmetry_looking__r_0.8,money__symmetry_looking__r_0.8500000000000001,money__symmetry_looking__r_0.9,money__symmetry_looking__r_0.9500000000000001,money__large_standard_deviation__r_0.05,money__large_standard_deviation__r_0.1,money__large_standard_deviation__r_0.15000000000000002,money__large_standard_deviation__r_0.2,money__large_standard_deviation__r_0.25,money__large_standard_deviation__r_0.30000000000000004,money__large_standard_deviation__r_0.35000000000000003,money__large_standard_deviation__r_0.4,money__large_standard_deviation__r_0.45,money__large_standard_deviation__r_0.5,money__large_standard_deviation__r_0.55,money__large_standard_deviation__r_0.6000000000000001,money__large_standard_deviation__r_0.65,money__large_standard_deviation__r_0.7000000000000001,money__large_standard_deviation__r_0.75,money__large_standard_deviation__r_0.8,money__large_standard_deviation__r_0.8500000000000001,money__large_standard_deviation__r_0.9,money__large_standard_deviation__r_0.9500000000000001,money__quantile__q_0.1,money__quantile__q_0.2,money__quantile__q_0.3,money__quantile__q_0.4,money__quantile__q_0.6,money__quantile__q_0.7,money__quantile__q_0.8,money__quantile__q_0.9,money__autocorrelation__lag_0,money__autocorrelation__lag_1,money__autocorrelation__lag_2,money__autocorrelation__lag_3,money__autocorrelation__lag_4,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__agg_autocorrelation__f_agg_""median""__maxlag_40","money__agg_autocorrelation__f_agg_""var""__maxlag_40",money__partial_autocorrelation__lag_0,money__partial_autocorrelation__lag_1,money__number_cwt_peaks__n_1,money__number_cwt_peaks__n_5,money__number_peaks__n_1,money__number_peaks__n_3,money__number_peaks__n_5,money__number_peaks__n_10,money__number_peaks__n_50,money__binned_entropy__max_bins_10,money__index_mass_quantile__q_0.1,money__index_mass_quantile__q_0.2,money__index_mass_quantile__q_0.3,money__index_mass_quantile__q_0.4,money__index_mass_quanti

In [39]:
df_test=merged1[merged1['m1']==0]
merged1.drop(merged1[merged1['m1']==0].index,inplace=True)

In [40]:
df_test.shape

(9336, 294)

In [41]:
merged1.shape

(28007, 294)

In [42]:
m1=merged1['m1']
m2=merged1['m2']
m3=merged1['m3']
m4=merged1['m4']
m5=merged1['m5']
m6=merged1['m6']

In [43]:
merged1.drop(columns=['m1','m2','m3','m4','m5','m6'],inplace=True)

In [44]:
df_test.drop(columns=['m1','m2','m3','m4','m5','m6'],inplace=True)

In [45]:
df_test.shape

(9336, 288)

In [46]:
merged1.shape

(28007, 288)

In [47]:
X_train=merged1.copy()

In [48]:
X_train.drop(columns=['ID', 'UpsellDate', 'ExpectedTermDate', 'LastPaymentDate', 'TransactionDates', 'PaymentsHistory','time'],inplace=True)

In [49]:
m1=m1*merged1['TotalContractValue']
m2=m2*merged1['TotalContractValue']
m3=m3*merged1['TotalContractValue']
m4=m4*merged1['TotalContractValue']
m5=m5*merged1['TotalContractValue']
m6=m6*merged1['TotalContractValue']

In [50]:

xtrain_m1, xval_m1, ytrain_m1, yval_m1 = train_test_split(
    X_train, 
    m1, 
    test_size=.3, 
    random_state=21, 
    shuffle=True
)
xtrain_m2, xval_m2, ytrain_m2, yval_m2 = train_test_split(
    X_train, 
    m2, 
    test_size=.3, 
    random_state=21, 
    shuffle=True
)
xtrain_m3, xval_m3, ytrain_m3, yval_m3 = train_test_split(
    X_train, 
    m3, 
    test_size=.3, 
    random_state=21, 
    shuffle=True
)
xtrain_m4, xval_m4, ytrain_m4, yval_m4 = train_test_split(
    X_train, 
    m4, 
    test_size=.3, 
    random_state=21, 
    shuffle=True
)
xtrain_m5, xval_m5, ytrain_m5, yval_m5 = train_test_split(
    X_train, 
    m5, 
    test_size=.3, 
    random_state=21, 
    shuffle=True
)
xtrain_m6, xval_m6, ytrain_m6, yval_m6 = train_test_split(
    X_train, 
    m6, 
    test_size=.3, 
    random_state=21, 
    shuffle=True
)

In [51]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
lgbm1=LGBMRegressor()
lgbm1.fit(xtrain_m1,ytrain_m1)
pred1=lgbm1.predict(xval_m1)
mse1=mean_squared_error(pred1,yval_m1)**0.5
print(mse1)

395.3638392903541


In [52]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
lgbm2=LGBMRegressor()
lgbm2.fit(xtrain_m2,ytrain_m2)
pred2=lgbm2.predict(xval_m2)
mse2=mean_squared_error(pred2,yval_m2)**0.5
print(mse2)

440.2042989535992


In [53]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
lgbm3=LGBMRegressor()
lgbm3.fit(xtrain_m3,ytrain_m3)
pred3=lgbm3.predict(xval_m3)
mse3=mean_squared_error(pred3,yval_m3)**0.5
print(mse3)

527.648214135509


In [54]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
lgbm4=LGBMRegressor()
lgbm4.fit(xtrain_m4,ytrain_m4)
pred4=lgbm4.predict(xval_m4)
mse4=mean_squared_error(pred4,yval_m4)**0.5
print(mse4)

454.11525158646407


In [55]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
lgbm5=LGBMRegressor()
lgbm5.fit(xtrain_m5,ytrain_m5)
pred5=lgbm5.predict(xval_m5)
mse5=mean_squared_error(pred5,yval_m5)**0.5
print(mse5)

920.6644900205646


In [56]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
lgbm6=LGBMRegressor()
lgbm6.fit(xtrain_m6,ytrain_m6)
pred6=lgbm6.predict(xval_m6)
mse6=mean_squared_error(pred6,yval_m6)**0.5
print(mse6)

1289.566025197475


In [57]:
subm=df_test[['ID']].copy()

In [58]:
subm.shape

(9336, 1)

In [59]:
subm['ID'].head()

1     ID_6L67PAA
5     ID_VJ80SX2
8     ID_7OU9HLK
9     ID_WVWTPGK
12    ID_04DSDQS
Name: ID, dtype: object

In [60]:
df_test.drop(columns=['ID', 'UpsellDate', 'ExpectedTermDate', 'LastPaymentDate', 'TransactionDates', 'PaymentsHistory','time'],inplace=True)

In [71]:
fin1=lgbm1.predict(df_test)
subm1=subm.copy()
subm1['Target']=fin1
subm1['ID']=subm1['ID']+' x m1'

In [74]:
fin2=lgbm2.predict(df_test)
subm2=subm.copy()
subm2['Target']=fin2
subm2['ID']=subm2['ID']+' x m2'

In [76]:
fin3=lgbm3.predict(df_test)
subm3=subm.copy()
subm3['Target']=fin3
subm3['ID']=subm3['ID']+' x m3'

In [77]:
fin4=lgbm4.predict(df_test)
subm4=subm.copy()
subm4['Target']=fin4
subm4['ID']=subm4['ID']+' x m4'

In [78]:
fin5=lgbm5.predict(df_test)
subm5=subm.copy()
subm5['Target']=fin5
subm5['ID']=subm5['ID']+' x m5'

In [79]:
fin6=lgbm6.predict(df_test)
subm6=subm.copy()
subm6['Target']=fin6
subm6['ID']=subm6['ID']+' x m6'

In [80]:
frames=[subm1,subm2,subm3,subm4,subm5,subm6]
submission=pd.concat(frames)

In [81]:
submission.sort_values(by='ID',inplace=True)

In [82]:
submission.head()

,ID,Target
4203,ID_000RHRU x m1,260.015533
4203,ID_000RHRU x m2,375.724108
4203,ID_000RHRU x m3,415.364648
4203,ID_000RHRU x m4,419.662794
4203,ID_000RHRU x m5,567.656847


In [83]:
submission.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
